In [2]:
# Parse the pasted TSV into a DataFrame, clean it, and run nonparametric tests
import io
import pandas as pd
import numpy as np
from scipy import stats

raw = """Prompt ID	Evaluator	Response Type	Dimension	Score
1	1	Large LLM	Comprehensiveness	0
1	1	Large LLM	Relevance	0
1	1	Large LLM	Specificity	0
1	1	Large LLM	Usefulness	0
1	1	Large LLM	Alignment	0
1	1	Small LLM	Comprehensiveness	9
1	1	Small LLM	Relevance	8
1	1	Small LLM	Specificity	6
1	1	Small LLM	Usefulness	8
1	1	Small LLM	Alignment	9
1	1	Human	Comprehensiveness	8
1	1	Human	Relevance	8
1	1	Human	Specificity	7
1	1	Human	Usefulness	7
1	1	Human	Alignment	9
2	1	Large LLM	Comprehensiveness	6
2	1	Large LLM	Relevance	6
2	1	Large LLM	Specificity	8
2	1	Large LLM	Usefulness	6
2	1	Large LLM	Alignment	9
2	1	Small LLM	Comprehensiveness	6
2	1	Small LLM	Relevance	7
2	1	Small LLM	Specificity	8
2	1	Small LLM	Usefulness	8
2	1	Small LLM	Alignment	8
2	1	Human	Comprehensiveness	7
2	1	Human	Relevance	7
2	1	Human	Specificity	7
2	1	Human	Usefulness	9
2	1	Human	Alignment	7
3	1	Large LLM	Comprehensiveness	8
3	1	Large LLM	Relevance	8
3	1	Large LLM	Specificity	7
3	1	Large LLM	Usefulness	8
3	1	Large LLM	Alignment	9
3	1	Small LLM	Comprehensiveness	8
3	1	Small LLM	Relevance	9
3	1	Small LLM	Specificity	9
3	1	Small LLM	Usefulness	8
3	1	Small LLM	Alignment	7
3	1	Human	Comprehensiveness	9
3	1	Human	Relevance	7
3	1	Human	Specificity	8
3	1	Human	Usefulness	9
3	1	Human	Alignment	9
4	1	Large LLM	Comprehensiveness	9
4	1	Large LLM	Relevance	8
4	1	Large LLM	Specificity	6
4	1	Large LLM	Usefulness	9
4	1	Large LLM	Alignment	8
4	1	Small LLM	Comprehensiveness	9
4	1	Small LLM	Relevance	7
4	1	Small LLM	Specificity	8
4	1	Small LLM	Usefulness	8
4	1	Small LLM	Alignment	8
4	1	Human	Comprehensiveness	8
4	1	Human	Relevance	8
4	1	Human	Specificity	8
4	1	Human	Usefulness	9
4	1	Human	Alignment	8
5	1	Large LLM	Comprehensiveness	9
5	1	Large LLM	Relevance	6
5	1	Large LLM	Specificity	6
5	1	Large LLM	Usefulness	9
5	1	Large LLM	Alignment	9
5	1	Small LLM	Comprehensiveness	9
5	1	Small LLM	Relevance	7
5	1	Small LLM	Specificity	6
5	1	Small LLM	Usefulness	9
5	1	Small LLM	Alignment	8
5	1	Human	Comprehensiveness	9
5	1	Human	Relevance	8
5	1	Human	Specificity	8
5	1	Human	Usefulness	8
5	1	Human	Alignment	8
6	1	Large LLM	Comprehensiveness	6
6	1	Large LLM	Relevance	8
6	1	Large LLM	Specificity	9
6	1	Large LLM	Usefulness	9
6	1	Large LLM	Alignment	9
6	1	Small LLM	Comprehensiveness	8
6	1	Small LLM	Relevance	8
6	1	Small LLM	Specificity	8
6	1	Small LLM	Usefulness	9
6	1	Small LLM	Alignment	9
6	1	Human	Comprehensiveness	6
6	1	Human	Relevance	9
6	1	Human	Specificity	9
6	1	Human	Usefulness	9
6	1	Human	Alignment	9
7	1	Large LLM	Comprehensiveness	8
7	1	Large LLM	Relevance	8
7	1	Large LLM	Specificity	9
7	1	Large LLM	Usefulness	9
7	1	Large LLM	Alignment	8
7	1	Small LLM	Comprehensiveness	8
7	1	Small LLM	Relevance	9
7	1	Small LLM	Specificity	8
7	1	Small LLM	Usefulness	7
7	1	Small LLM	Alignment	
7	1	Human	Comprehensiveness	9
7	1	Human	Relevance	6
7	1	Human	Specificity	9
7	1	Human	Usefulness	8
7	1	Human	Alignment	8
8	1	Large LLM	Comprehensiveness	8
8	1	Large LLM	Relevance	9
8	1	Large LLM	Specificity	8
8	1	Large LLM	Usefulness	8
8	1	Large LLM	Alignment	6
8	1	Small LLM	Comprehensiveness	7
8	1	Small LLM	Relevance	9
8	1	Small LLM	Specificity	7
8	1	Small LLM	Usefulness	7
8	1	Small LLM	Alignment	8
8	1	Human	Comprehensiveness	7
8	1	Human	Relevance	8
8	1	Human	Specificity	8
8	1	Human	Usefulness	7
8	1	Human	Alignment	9
9	1	Large LLM	Comprehensiveness	6
9	1	Large LLM	Relevance	8
9	1	Large LLM	Specificity	7
9	1	Large LLM	Usefulness	7
9	1	Large LLM	Alignment	8
9	1	Small LLM	Comprehensiveness	7
9	1	Small LLM	Relevance	6
9	1	Small LLM	Specificity	7
9	1	Small LLM	Usefulness	5
9	1	Small LLM	Alignment	8
9	1	Human	Comprehensiveness	8
9	1	Human	Relevance	8
9	1	Human	Specificity	7
9	1	Human	Usefulness	7
9	1	Human	Alignment	8
10	1	Large LLM	Comprehensiveness	7
10	1	Large LLM	Relevance	8
10	1	Large LLM	Specificity	8
10	1	Large LLM	Usefulness	6
10	1	Large LLM	Alignment	7
10	1	Small LLM	Comprehensiveness	8
10	1	Small LLM	Relevance	8
10	1	Small LLM	Specificity	7
10	1	Small LLM	Usefulness	6
10	1	Small LLM	Alignment	9
10	1	Human	Comprehensiveness	8
10	1	Human	Relevance	8
10	1	Human	Specificity	7
10	1	Human	Usefulness	7
10	1	Human	Alignment	7
11	1	Large LLM	Comprehensiveness	5
11	1	Large LLM	Relevance	9
11	1	Large LLM	Specificity	8
11	1	Large LLM	Usefulness	8
11	1	Large LLM	Alignment	8
11	1	Small LLM	Comprehensiveness	8
11	1	Small LLM	Relevance	7
11	1	Small LLM	Specificity	9
11	1	Small LLM	Usefulness	8
11	1	Small LLM	Alignment	6
11	1	Human	Comprehensiveness	7
11	1	Human	Relevance	7
11	1	Human	Specificity	8
11	1	Human	Usefulness	8
11	1	Human	Alignment	7
12	1	Large LLM	Comprehensiveness	8
12	1	Large LLM	Relevance	7
12	1	Large LLM	Specificity	8
12	1	Large LLM	Usefulness	6
12	1	Large LLM	Alignment	8
12	1	Small LLM	Comprehensiveness	8
12	1	Small LLM	Relevance	7
12	1	Small LLM	Specificity	6
12	1	Small LLM	Usefulness	8
12	1	Small LLM	Alignment	8
12	1	Human	Comprehensiveness	7
12	1	Human	Relevance	8
12	1	Human	Specificity	7
12	1	Human	Usefulness	6
12	1	Human	Alignment	8
13	1	Large LLM	Comprehensiveness	8
13	1	Large LLM	Relevance	8
13	1	Large LLM	Specificity	7
13	1	Large LLM	Usefulness	7
13	1	Large LLM	Alignment	9
13	1	Small LLM	Comprehensiveness	8
13	1	Small LLM	Relevance	7
13	1	Small LLM	Specificity	6
13	1	Small LLM	Usefulness	6
13	1	Small LLM	Alignment	8
13	1	Human	Comprehensiveness	8
13	1	Human	Relevance	9
13	1	Human	Specificity	9
13	1	Human	Usefulness	9
13	1	Human	Alignment	8
14	1	Large LLM	Comprehensiveness	9
14	1	Large LLM	Relevance	8
14	1	Large LLM	Specificity	7
14	1	Large LLM	Usefulness	6
14	1	Large LLM	Alignment	7
14	1	Small LLM	Comprehensiveness	9
14	1	Small LLM	Relevance	8
14	1	Small LLM	Specificity	7
14	1	Small LLM	Usefulness	6
14	1	Small LLM	Alignment	8
14	1	Human	Comprehensiveness	7
14	1	Human	Relevance	8
14	1	Human	Specificity	8
14	1	Human	Usefulness	5
14	1	Human	Alignment	8
1	2	Large LLM	Comprehensiveness	10
1	2	Large LLM	Relevance	10
1	2	Large LLM	Specificity	9
1	2	Large LLM	Usefulness	10
1	2	Large LLM	Alignment	10
1	2	Small LLM	Comprehensiveness	10
1	2	Small LLM	Relevance	9
1	2	Small LLM	Specificity	9
1	2	Small LLM	Usefulness	9
1	2	Small LLM	Alignment	9
1	2	Human	Comprehensiveness	8
1	2	Human	Relevance	8
1	2	Human	Specificity	7
1	2	Human	Usefulness	8
1	2	Human	Alignment	8
2	2	Large LLM	Comprehensiveness	10
2	2	Large LLM	Relevance	9
2	2	Large LLM	Specificity	10
2	2	Large LLM	Usefulness	7
2	2	Large LLM	Alignment	8
2	2	Small LLM	Comprehensiveness	10
2	2	Small LLM	Relevance	10
2	2	Small LLM	Specificity	10
2	2	Small LLM	Usefulness	9
2	2	Small LLM	Alignment	9
2	2	Human	Comprehensiveness	9
2	2	Human	Relevance	10
2	2	Human	Specificity	10
2	2	Human	Usefulness	8
2	2	Human	Alignment	9
3	2	Large LLM	Comprehensiveness	10
3	2	Large LLM	Relevance	10
3	2	Large LLM	Specificity	9
3	2	Large LLM	Usefulness	9
3	2	Large LLM	Alignment	9
3	2	Small LLM	Comprehensiveness	9
3	2	Small LLM	Relevance	8
3	2	Small LLM	Specificity	8
3	2	Small LLM	Usefulness	8
3	2	Small LLM	Alignment	8
3	2	Human	Comprehensiveness	8
3	2	Human	Relevance	8
3	2	Human	Specificity	7
3	2	Human	Usefulness	7
3	2	Human	Alignment	7
4	2	Large LLM	Comprehensiveness	8
4	2	Large LLM	Relevance	9
4	2	Large LLM	Specificity	8
4	2	Large LLM	Usefulness	8
4	2	Large LLM	Alignment	8
4	2	Small LLM	Comprehensiveness	10
4	2	Small LLM	Relevance	10
4	2	Small LLM	Specificity	10
4	2	Small LLM	Usefulness	10
4	2	Small LLM	Alignment	10
4	2	Human	Comprehensiveness	6
4	2	Human	Relevance	9
4	2	Human	Specificity	7
4	2	Human	Usefulness	8
4	2	Human	Alignment	8
5	2	Large LLM	Comprehensiveness	8
5	2	Large LLM	Relevance	9
5	2	Large LLM	Specificity	7
5	2	Large LLM	Usefulness	8
5	2	Large LLM	Alignment	8
5	2	Small LLM	Comprehensiveness	8
5	2	Small LLM	Relevance	8
5	2	Small LLM	Specificity	8
5	2	Small LLM	Usefulness	9
5	2	Small LLM	Alignment	9
5	2	Human	Comprehensiveness	8
5	2	Human	Relevance	9
5	2	Human	Specificity	8
5	2	Human	Usefulness	8
5	2	Human	Alignment	8
6	2	Large LLM	Comprehensiveness	8
6	2	Large LLM	Relevance	10
6	2	Large LLM	Specificity	8
6	2	Large LLM	Usefulness	10
6	2	Large LLM	Alignment	10
6	2	Small LLM	Comprehensiveness	10
6	2	Small LLM	Relevance	10
6	2	Small LLM	Specificity	9
6	2	Small LLM	Usefulness	9
6	2	Small LLM	Alignment	10
6	2	Human	Comprehensiveness	9
6	2	Human	Relevance	8
6	2	Human	Specificity	8
6	2	Human	Usefulness	8
6	2	Human	Alignment	8
7	2	Large LLM	Comprehensiveness	9
7	2	Large LLM	Relevance	8
7	2	Large LLM	Specificity	8
7	2	Large LLM	Usefulness	8
7	2	Large LLM	Alignment	9
7	2	Small LLM	Comprehensiveness	9
7	2	Small LLM	Relevance	9
7	2	Small LLM	Specificity	9
7	2	Small LLM	Usefulness	9
7	2	Small LLM	Alignment	9
7	2	Human	Comprehensiveness	9
7	2	Human	Relevance	9
7	2	Human	Specificity	9
7	2	Human	Usefulness	9
7	2	Human	Alignment	10
8	2	Large LLM	Comprehensiveness	7
8	2	Large LLM	Relevance	9
8	2	Large LLM	Specificity	7
8	2	Large LLM	Usefulness	9
8	2	Large LLM	Alignment	9
8	2	Small LLM	Comprehensiveness	7
8	2	Small LLM	Relevance	8
8	2	Small LLM	Specificity	8
8	2	Small LLM	Usefulness	8
8	2	Small LLM	Alignment	8
8	2	Human	Comprehensiveness	9
8	2	Human	Relevance	8
8	2	Human	Specificity	8
8	2	Human	Usefulness	8
8	2	Human	Alignment	9
9	2	Large LLM	Comprehensiveness	10
9	2	Large LLM	Relevance	9
9	2	Large LLM	Specificity	9
9	2	Large LLM	Usefulness	9
9	2	Large LLM	Alignment	9
9	2	Small LLM	Comprehensiveness	10
9	2	Small LLM	Relevance	9
9	2	Small LLM	Specificity	9
9	2	Small LLM	Usefulness	9
9	2	Small LLM	Alignment	9
9	2	Human	Comprehensiveness	10
9	2	Human	Relevance	10
9	2	Human	Specificity	9
9	2	Human	Usefulness	9
9	2	Human	Alignment	9
10	2	Large LLM	Comprehensiveness	8
10	2	Large LLM	Relevance	8
10	2	Large LLM	Specificity	7
10	2	Large LLM	Usefulness	7
10	2	Large LLM	Alignment	7
10	2	Small LLM	Comprehensiveness	9
10	2	Small LLM	Relevance	8
10	2	Small LLM	Specificity	8
10	2	Small LLM	Usefulness	8
10	2	Small LLM	Alignment	8
10	2	Human	Comprehensiveness	8
10	2	Human	Relevance	8
10	2	Human	Specificity	8
10	2	Human	Usefulness	8
10	2	Human	Alignment	8
11	2	Large LLM	Comprehensiveness	10
11	2	Large LLM	Relevance	8
11	2	Large LLM	Specificity	7
11	2	Large LLM	Usefulness	7
11	2	Large LLM	Alignment	7
11	2	Small LLM	Comprehensiveness	9
11	2	Small LLM	Relevance	8
11	2	Small LLM	Specificity	8
11	2	Small LLM	Usefulness	8
11	2	Small LLM	Alignment	9
11	2	Human	Comprehensiveness	10
11	2	Human	Relevance	9
11	2	Human	Specificity	9
11	2	Human	Usefulness	8
11	2	Human	Alignment	9
12	2	Large LLM	Comprehensiveness	10
12	2	Large LLM	Relevance	10
12	2	Large LLM	Specificity	9
12	2	Large LLM	Usefulness	9
12	2	Large LLM	Alignment	10
12	2	Small LLM	Comprehensiveness	9
12	2	Small LLM	Relevance	8
12	2	Small LLM	Specificity	8
12	2	Small LLM	Usefulness	8
12	2	Small LLM	Alignment	9
12	2	Human	Comprehensiveness	10
12	2	Human	Relevance	10
12	2	Human	Specificity	9
12	2	Human	Usefulness	9
12	2	Human	Alignment	10
13	2	Large LLM	Comprehensiveness	9
13	2	Large LLM	Relevance	10
13	2	Large LLM	Specificity	9
13	2	Large LLM	Usefulness	9
13	2	Large LLM	Alignment	9
13	2	Small LLM	Comprehensiveness	10
13	2	Small LLM	Relevance	10
13	2	Small LLM	Specificity	10
13	2	Small LLM	Usefulness	9
13	2	Small LLM	Alignment	9
13	2	Human	Comprehensiveness	9
13	2	Human	Relevance	9
13	2	Human	Specificity	9
13	2	Human	Usefulness	9
13	2	Human	Alignment	9
14	2	Large LLM	Comprehensiveness	7
14	2	Large LLM	Relevance	8
14	2	Large LLM	Specificity	8
14	2	Large LLM	Usefulness	8
14	2	Large LLM	Alignment	8
14	2	Small LLM	Comprehensiveness	8
14	2	Small LLM	Relevance	8
14	2	Small LLM	Specificity	8
14	2	Small LLM	Usefulness	8
14	2	Small LLM	Alignment	8
14	2	Human	Comprehensiveness	9
14	2	Human	Relevance	9
14	2	Human	Specificity	8
14	2	Human	Usefulness	8
14	2	Human	Alignment	8
1	3	Large LLM	Comprehensiveness	10
1	3	Large LLM	Relevance	10
1	3	Large LLM	Specificity	8
1	3	Large LLM	Usefulness	10
1	3	Large LLM	Alignment	10
1	3	Small LLM	Comprehensiveness	8
1	3	Small LLM	Relevance	8
1	3	Small LLM	Specificity	10
1	3	Small LLM	Usefulness	10
1	3	Small LLM	Alignment	10
1	3	Human	Comprehensiveness	5
1	3	Human	Relevance	5
1	3	Human	Specificity	5
1	3	Human	Usefulness	5
1	3	Human	Alignment	5
2	3	Large LLM	Comprehensiveness	8
2	3	Large LLM	Relevance	8
2	3	Large LLM	Specificity	8
2	3	Large LLM	Usefulness	8
2	3	Large LLM	Alignment	6
2	3	Small LLM	Comprehensiveness	5
2	3	Small LLM	Relevance	10
2	3	Small LLM	Specificity	5
2	3	Small LLM	Usefulness	8
2	3	Small LLM	Alignment	7
2	3	Human	Comprehensiveness	7
2	3	Human	Relevance	9
2	3	Human	Specificity	8
2	3	Human	Usefulness	10
2	3	Human	Alignment	10
3	3	Large LLM	Comprehensiveness	8
3	3	Large LLM	Relevance	8
3	3	Large LLM	Specificity	7
3	3	Large LLM	Usefulness	9
3	3	Large LLM	Alignment	8
3	3	Small LLM	Comprehensiveness	7
3	3	Small LLM	Relevance	7
3	3	Small LLM	Specificity	7
3	3	Small LLM	Usefulness	7
3	3	Small LLM	Alignment	7
3	3	Human	Comprehensiveness	5
3	3	Human	Relevance	5
3	3	Human	Specificity	5
3	3	Human	Usefulness	5
3	3	Human	Alignment	5
4	3	Large LLM	Comprehensiveness	10
4	3	Large LLM	Relevance	10
4	3	Large LLM	Specificity	10
4	3	Large LLM	Usefulness	10
4	3	Large LLM	Alignment	10
4	3	Small LLM	Comprehensiveness	8
4	3	Small LLM	Relevance	7
4	3	Small LLM	Specificity	6
4	3	Small LLM	Usefulness	8
4	3	Small LLM	Alignment	7
4	3	Human	Comprehensiveness	7
4	3	Human	Relevance	8
4	3	Human	Specificity	6
4	3	Human	Usefulness	8
4	3	Human	Alignment	7
5	3	Large LLM	Comprehensiveness	10
5	3	Large LLM	Relevance	8
5	3	Large LLM	Specificity	9
5	3	Large LLM	Usefulness	9
5	3	Large LLM	Alignment	9
5	3	Small LLM	Comprehensiveness	10
5	3	Small LLM	Relevance	8
5	3	Small LLM	Specificity	8
5	3	Small LLM	Usefulness	10
5	3	Small LLM	Alignment	9
5	3	Human	Comprehensiveness	5
5	3	Human	Relevance	7
5	3	Human	Specificity	5
5	3	Human	Usefulness	5
5	3	Human	Alignment	6
6	3	Large LLM	Comprehensiveness	10
6	3	Large LLM	Relevance	9
6	3	Large LLM	Specificity	9
6	3	Large LLM	Usefulness	10
6	3	Large LLM	Alignment	10
6	3	Small LLM	Comprehensiveness	6
6	3	Small LLM	Relevance	7
6	3	Small LLM	Specificity	6
6	3	Small LLM	Usefulness	7
6	3	Small LLM	Alignment	7
6	3	Human	Comprehensiveness	3
6	3	Human	Relevance	3
6	3	Human	Specificity	3
6	3	Human	Usefulness	2
6	3	Human	Alignment	2
7	3	Large LLM	Comprehensiveness	10
7	3	Large LLM	Relevance	10
7	3	Large LLM	Specificity	10
7	3	Large LLM	Usefulness	10
7	3	Large LLM	Alignment	10
7	3	Small LLM	Comprehensiveness	10
7	3	Small LLM	Relevance	10
7	3	Small LLM	Specificity	7
7	3	Small LLM	Usefulness	8
7	3	Small LLM	Alignment	7
7	3	Human	Comprehensiveness	6
7	3	Human	Relevance	8
7	3	Human	Specificity	6
7	3	Human	Usefulness	7
7	3	Human	Alignment	7
8	3	Large LLM	Comprehensiveness	9
8	3	Large LLM	Relevance	9
8	3	Large LLM	Specificity	10
8	3	Large LLM	Usefulness	9
8	3	Large LLM	Alignment	9
8	3	Small LLM	Comprehensiveness	8
8	3	Small LLM	Relevance	7
8	3	Small LLM	Specificity	7
8	3	Small LLM	Usefulness	8
8	3	Small LLM	Alignment	8
8	3	Human	Comprehensiveness	7
8	3	Human	Relevance	6
8	3	Human	Specificity	6
8	3	Human	Usefulness	7
8	3	Human	Alignment	7
9	3	Large LLM	Comprehensiveness	10
9	3	Large LLM	Relevance	10
9	3	Large LLM	Specificity	10
9	3	Large LLM	Usefulness	10
9	3	Large LLM	Alignment	10
9	3	Small LLM	Comprehensiveness	8
9	3	Small LLM	Relevance	8
9	3	Small LLM	Specificity	7
9	3	Small LLM	Usefulness	8
9	3	Small LLM	Alignment	7
9	3	Human	Comprehensiveness	5
9	3	Human	Relevance	5
9	3	Human	Specificity	5
9	3	Human	Usefulness	6
9	3	Human	Alignment	6
10	3	Large LLM	Comprehensiveness	9
10	3	Large LLM	Relevance	9
10	3	Large LLM	Specificity	9
10	3	Large LLM	Usefulness	9
10	3	Large LLM	Alignment	9
10	3	Small LLM	Comprehensiveness	6
10	3	Small LLM	Relevance	6
10	3	Small LLM	Specificity	6
10	3	Small LLM	Usefulness	7
10	3	Small LLM	Alignment	7
10	3	Human	Comprehensiveness	6
10	3	Human	Relevance	6
10	3	Human	Specificity	4
10	3	Human	Usefulness	6
10	3	Human	Alignment	5
11	3	Large LLM	Comprehensiveness	9
11	3	Large LLM	Relevance	9
11	3	Large LLM	Specificity	9
11	3	Large LLM	Usefulness	9
11	3	Large LLM	Alignment	9
11	3	Small LLM	Comprehensiveness	7
11	3	Small LLM	Relevance	7
11	3	Small LLM	Specificity	7
11	3	Small LLM	Usefulness	8
11	3	Small LLM	Alignment	7
11	3	Human	Comprehensiveness	6
11	3	Human	Relevance	5
11	3	Human	Specificity	6
11	3	Human	Usefulness	5
11	3	Human	Alignment	6
12	3	Large LLM	Comprehensiveness	9
12	3	Large LLM	Relevance	10
12	3	Large LLM	Specificity	10
12	3	Large LLM	Usefulness	9
12	3	Large LLM	Alignment	10
12	3	Small LLM	Comprehensiveness	10
12	3	Small LLM	Relevance	7
12	3	Small LLM	Specificity	7
12	3	Small LLM	Usefulness	6
12	3	Small LLM	Alignment	7
12	3	Human	Comprehensiveness	7
12	3	Human	Relevance	7
12	3	Human	Specificity	7
12	3	Human	Usefulness	7
12	3	Human	Alignment	7
13	3	Large LLM	Comprehensiveness	10
13	3	Large LLM	Relevance	10
13	3	Large LLM	Specificity	10
13	3	Large LLM	Usefulness	10
13	3	Large LLM	Alignment	10
13	3	Small LLM	Comprehensiveness	5
13	3	Small LLM	Relevance	5
13	3	Small LLM	Specificity	5
13	3	Small LLM	Usefulness	7
13	3	Small LLM	Alignment	5
13	3	Human	Comprehensiveness	6
13	3	Human	Relevance	6
13	3	Human	Specificity	6
13	3	Human	Usefulness	6
13	3	Human	Alignment	5
14	3	Large LLM	Comprehensiveness	9
14	3	Large LLM	Relevance	9
14	3	Large LLM	Specificity	9
14	3	Large LLM	Usefulness	9
14	3	Large LLM	Alignment	9
14	3	Small LLM	Comprehensiveness	8
14	3	Small LLM	Relevance	8
14	3	Small LLM	Specificity	7
14	3	Small LLM	Usefulness	8
14	3	Small LLM	Alignment	7
14	3	Human	Comprehensiveness	6
14	3	Human	Relevance	5
14	3	Human	Specificity	5
14	3	Human	Usefulness	6
14	3	Human	Alignment	5
1	4	Large LLM	Comprehensiveness	8
1	4	Large LLM	Relevance	9
1	4	Large LLM	Specificity	7
1	4	Large LLM	Usefulness	8
1	4	Large LLM	Alignment	8
1	4	Small LLM	Comprehensiveness	7
1	4	Small LLM	Relevance	6
1	4	Small LLM	Specificity	8
1	4	Small LLM	Usefulness	9
1	4	Small LLM	Alignment	9
1	4	Human	Comprehensiveness	6
1	4	Human	Relevance	7
1	4	Human	Specificity	9
1	4	Human	Usefulness	7
1	4	Human	Alignment	7
2	4	Large LLM	Comprehensiveness	6
2	4	Large LLM	Relevance	9
2	4	Large LLM	Specificity	7
2	4	Large LLM	Usefulness	7
2	4	Large LLM	Alignment	7
2	4	Small LLM	Comprehensiveness	8
2	4	Small LLM	Relevance	9
2	4	Small LLM	Specificity	7
2	4	Small LLM	Usefulness	7
2	4	Small LLM	Alignment	8
2	4	Human	Comprehensiveness	9
2	4	Human	Relevance	8
2	4	Human	Specificity	5
2	4	Human	Usefulness	4
2	4	Human	Alignment	9
3	4	Large LLM	Comprehensiveness	7
3	4	Large LLM	Relevance	8
3	4	Large LLM	Specificity	8
3	4	Large LLM	Usefulness	8
3	4	Large LLM	Alignment	7
3	4	Small LLM	Comprehensiveness	8
3	4	Small LLM	Relevance	9
3	4	Small LLM	Specificity	8
3	4	Small LLM	Usefulness	7
3	4	Small LLM	Alignment	8
3	4	Human	Comprehensiveness	6
3	4	Human	Relevance	8
3	4	Human	Specificity	9
3	4	Human	Usefulness	9
3	4	Human	Alignment	10
4	4	Large LLM	Comprehensiveness	8
4	4	Large LLM	Relevance	8
4	4	Large LLM	Specificity	7
4	4	Large LLM	Usefulness	5
4	4	Large LLM	Alignment	9
4	4	Small LLM	Comprehensiveness	7
4	4	Small LLM	Relevance	9
4	4	Small LLM	Specificity	9
4	4	Small LLM	Usefulness	9
4	4	Small LLM	Alignment	7
4	4	Human	Comprehensiveness	7
4	4	Human	Relevance	6
4	4	Human	Specificity	9
4	4	Human	Usefulness	9
4	4	Human	Alignment	9
5	4	Large LLM	Comprehensiveness	0
5	4	Large LLM	Relevance	6
5	4	Large LLM	Specificity	6
5	4	Large LLM	Usefulness	9
5	4	Large LLM	Alignment	9
5	4	Small LLM	Comprehensiveness	9
5	4	Small LLM	Relevance	7
5	4	Small LLM	Specificity	6
5	4	Small LLM	Usefulness	9
5	4	Small LLM	Alignment	8
5	4	Human	Comprehensiveness	9
5	4	Human	Relevance	8
5	4	Human	Specificity	8
5	4	Human	Usefulness	8
5	4	Human	Alignment	8
6	4	Large LLM	Comprehensiveness	6
6	4	Large LLM	Relevance	8
6	4	Large LLM	Specificity	9
6	4	Large LLM	Usefulness	9
6	4	Large LLM	Alignment	9
6	4	Small LLM	Comprehensiveness	8
6	4	Small LLM	Relevance	8
6	4	Small LLM	Specificity	8
6	4	Small LLM	Usefulness	9
6	4	Small LLM	Alignment	9
6	4	Human	Comprehensiveness	6
6	4	Human	Relevance	9
6	4	Human	Specificity	9
6	4	Human	Usefulness	9
6	4	Human	Alignment	9
7	4	Large LLM	Comprehensiveness	8
7	4	Large LLM	Relevance	8
7	4	Large LLM	Specificity	9
7	4	Large LLM	Usefulness	9
7	4	Large LLM	Alignment	8
7	4	Small LLM	Comprehensiveness	8
7	4	Small LLM	Relevance	9
7	4	Small LLM	Specificity	8
7	4	Small LLM	Usefulness	7
7	4	Small LLM	Alignment	
7	4	Human	Comprehensiveness	9
7	4	Human	Relevance	6
7	4	Human	Specificity	9
7	4	Human	Usefulness	8
7	4	Human	Alignment	8
8	4	Large LLM	Comprehensiveness	8
8	4	Large LLM	Relevance	9
8	4	Large LLM	Specificity	8
8	4	Large LLM	Usefulness	8
8	4	Large LLM	Alignment	6
8	4	Small LLM	Comprehensiveness	7
8	4	Small LLM	Relevance	9
8	4	Small LLM	Specificity	7
8	4	Small LLM	Usefulness	7
8	4	Small LLM	Alignment	8
8	4	Human	Comprehensiveness	7
8	4	Human	Relevance	8
8	4	Human	Specificity	8
8	4	Human	Usefulness	7
8	4	Human	Alignment	9
9	4	Large LLM	Comprehensiveness	6
9	4	Large LLM	Relevance	8
9	4	Large LLM	Specificity	7
9	4	Large LLM	Usefulness	7
9	4	Large LLM	Alignment	8
9	4	Small LLM	Comprehensiveness	7
9	4	Small LLM	Relevance	6
9	4	Small LLM	Specificity	7
9	4	Small LLM	Usefulness	5
9	4	Small LLM	Alignment	8
9	4	Human	Comprehensiveness	8
9	4	Human	Relevance	8
9	4	Human	Specificity	7
9	4	Human	Usefulness	7
9	4	Human	Alignment	8
10	4	Large LLM	Comprehensiveness	8
10	4	Large LLM	Relevance	9
10	4	Large LLM	Specificity	9
10	4	Large LLM	Usefulness	8
10	4	Large LLM	Alignment	8
10	4	Small LLM	Comprehensiveness	9
10	4	Small LLM	Relevance	8
10	4	Small LLM	Specificity	7
10	4	Small LLM	Usefulness	8
10	4	Small LLM	Alignment	7
10	4	Human	Comprehensiveness	8
10	4	Human	Relevance	8
10	4	Human	Specificity	8
10	4	Human	Usefulness	8
10	4	Human	Alignment	8
11	4	Large LLM	Comprehensiveness	10
11	4	Large LLM	Relevance	8
11	4	Large LLM	Specificity	9
11	4	Large LLM	Usefulness	10
11	4	Large LLM	Alignment	10
11	4	Small LLM	Comprehensiveness	8
11	4	Small LLM	Relevance	7
11	4	Small LLM	Specificity	8
11	4	Small LLM	Usefulness	7
11	4	Small LLM	Alignment	8
11	4	Human	Comprehensiveness	8
11	4	Human	Relevance	7
11	4	Human	Specificity	6
11	4	Human	Usefulness	7
11	4	Human	Alignment	7
12	4	Large LLM	Comprehensiveness	8
12	4	Large LLM	Relevance	7
12	4	Large LLM	Specificity	8
12	4	Large LLM	Usefulness	6
12	4	Large LLM	Alignment	8
12	4	Small LLM	Comprehensiveness	8
12	4	Small LLM	Relevance	7
12	4	Small LLM	Specificity	6
12	4	Small LLM	Usefulness	8
12	4	Small LLM	Alignment	8
12	4	Human	Comprehensiveness	7
12	4	Human	Relevance	8
12	4	Human	Specificity	7
12	4	Human	Usefulness	6
12	4	Human	Alignment	8
13	4	Large LLM	Comprehensiveness	7
13	4	Large LLM	Relevance	9
13	4	Large LLM	Specificity	8
13	4	Large LLM	Usefulness	8
13	4	Large LLM	Alignment	9
13	4	Small LLM	Comprehensiveness	8
13	4	Small LLM	Relevance	8
13	4	Small LLM	Specificity	7
13	4	Small LLM	Usefulness	8
13	4	Small LLM	Alignment	7
13	4	Human	Comprehensiveness	7
13	4	Human	Relevance	7
13	4	Human	Specificity	7
13	4	Human	Usefulness	8
13	4	Human	Alignment	7
14	4	Large LLM	Comprehensiveness	9
14	4	Large LLM	Relevance	8
14	4	Large LLM	Specificity	9
14	4	Large LLM	Usefulness	9
14	4	Large LLM	Alignment	9
14	4	Small LLM	Comprehensiveness	8
14	4	Small LLM	Relevance	8
14	4	Small LLM	Specificity	7
14	4	Small LLM	Usefulness	8
14	4	Small LLM	Alignment	8
14	4	Human	Comprehensiveness	7
14	4	Human	Relevance	8
14	4	Human	Specificity	7
14	4	Human	Usefulness	7
14	4	Human	Alignment	7
1	5	Large LLM	Comprehensiveness	8
1	5	Large LLM	Relevance	9
1	5	Large LLM	Specificity	8
1	5	Large LLM	Usefulness	9
1	5	Large LLM	Alignment	8
1	5	Small LLM	Comprehensiveness	8
1	5	Small LLM	Relevance	7
1	5	Small LLM	Specificity	6
1	5	Small LLM	Usefulness	7
1	5	Small LLM	Alignment	9
1	5	Human	Comprehensiveness	9
1	5	Human	Relevance	7
1	5	Human	Specificity	6
1	5	Human	Usefulness	6
1	5	Human	Alignment	7
2	5	Large LLM	Comprehensiveness	9
2	5	Large LLM	Relevance	8
2	5	Large LLM	Specificity	10
2	5	Large LLM	Usefulness	10
2	5	Large LLM	Alignment	9
2	5	Small LLM	Comprehensiveness	9
2	5	Small LLM	Relevance	8
2	5	Small LLM	Specificity	6
2	5	Small LLM	Usefulness	5
2	5	Small LLM	Alignment	8
2	5	Human	Comprehensiveness	8
2	5	Human	Relevance	9
2	5	Human	Specificity	9
2	5	Human	Usefulness	8
2	5	Human	Alignment	7
3	5	Large LLM	Comprehensiveness	8
3	5	Large LLM	Relevance	9
3	5	Large LLM	Specificity	8
3	5	Large LLM	Usefulness	8
3	5	Large LLM	Alignment	8
3	5	Small LLM	Comprehensiveness	8
3	5	Small LLM	Relevance	7
3	5	Small LLM	Specificity	5
3	5	Small LLM	Usefulness	6
3	5	Small LLM	Alignment	6
3	5	Human	Comprehensiveness	5
3	5	Human	Relevance	7
3	5	Human	Specificity	7
3	5	Human	Usefulness	7
3	5	Human	Alignment	7
4	5	Large LLM	Comprehensiveness	8
4	5	Large LLM	Relevance	10
4	5	Large LLM	Specificity	10
4	5	Large LLM	Usefulness	10
4	5	Large LLM	Alignment	10
4	5	Small LLM	Comprehensiveness	7
4	5	Small LLM	Relevance	8
4	5	Small LLM	Specificity	8
4	5	Small LLM	Usefulness	8
4	5	Small LLM	Alignment	8
4	5	Human	Comprehensiveness	9
4	5	Human	Relevance	6
4	5	Human	Specificity	6
4	5	Human	Usefulness	7
4	5	Human	Alignment	6
5	5	Large LLM	Comprehensiveness	10
5	5	Large LLM	Relevance	9
5	5	Large LLM	Specificity	9
5	5	Large LLM	Usefulness	8
5	5	Large LLM	Alignment	10
5	5	Small LLM	Comprehensiveness	8
5	5	Small LLM	Relevance	8
5	5	Small LLM	Specificity	8
5	5	Small LLM	Usefulness	8
5	5	Small LLM	Alignment	8
5	5	Human	Comprehensiveness	8
5	5	Human	Relevance	8
5	5	Human	Specificity	7
5	5	Human	Usefulness	8
5	5	Human	Alignment	8
6	5	Large LLM	Comprehensiveness	9
6	5	Large LLM	Relevance	10
6	5	Large LLM	Specificity	9
6	5	Large LLM	Usefulness	7
6	5	Large LLM	Alignment	8
6	5	Small LLM	Comprehensiveness	9
6	5	Small LLM	Relevance	8
6	5	Small LLM	Specificity	9
6	5	Small LLM	Usefulness	7
6	5	Small LLM	Alignment	8
6	5	Human	Comprehensiveness	7
6	5	Human	Relevance	8
6	5	Human	Specificity	5
6	5	Human	Usefulness	7
6	5	Human	Alignment	8
7	5	Large LLM	Comprehensiveness	9
7	5	Large LLM	Relevance	9
7	5	Large LLM	Specificity	7
7	5	Large LLM	Usefulness	8
7	5	Large LLM	Alignment	9
7	5	Small LLM	Comprehensiveness	9
7	5	Small LLM	Relevance	8
7	5	Small LLM	Specificity	7
7	5	Small LLM	Usefulness	6
7	5	Small LLM	Alignment	8
7	5	Human	Comprehensiveness	6
7	5	Human	Relevance	9
7	5	Human	Specificity	7
7	5	Human	Usefulness	7
7	5	Human	Alignment	7
8	5	Large LLM	Comprehensiveness	8
8	5	Large LLM	Relevance	8
8	5	Large LLM	Specificity	10
8	5	Large LLM	Usefulness	9
8	5	Large LLM	Alignment	9
8	5	Small LLM	Comprehensiveness	9
8	5	Small LLM	Relevance	9
8	5	Small LLM	Specificity	8
8	5	Small LLM	Usefulness	9
8	5	Small LLM	Alignment	10
8	5	Human	Comprehensiveness	7
8	5	Human	Relevance	7
8	5	Human	Specificity	7
8	5	Human	Usefulness	8
8	5	Human	Alignment	7
9	5	Large LLM	Comprehensiveness	7
9	5	Large LLM	Relevance	8
9	5	Large LLM	Specificity	8
9	5	Large LLM	Usefulness	10
9	5	Large LLM	Alignment	8
9	5	Small LLM	Comprehensiveness	8
9	5	Small LLM	Relevance	7
9	5	Small LLM	Specificity	7
9	5	Small LLM	Usefulness	8
9	5	Small LLM	Alignment	7
9	5	Human	Comprehensiveness	6
9	5	Human	Relevance	8
9	5	Human	Specificity	6
9	5	Human	Usefulness	7
9	5	Human	Alignment	8
10	5	Large LLM	Comprehensiveness	8
10	5	Large LLM	Relevance	9
10	5	Large LLM	Specificity	9
10	5	Large LLM	Usefulness	8
10	5	Large LLM	Alignment	8
10	5	Small LLM	Comprehensiveness	9
10	5	Small LLM	Relevance	8
10	5	Small LLM	Specificity	7
10	5	Small LLM	Usefulness	8
10	5	Small LLM	Alignment	7
10	5	Human	Comprehensiveness	8
10	5	Human	Relevance	8
10	5	Human	Specificity	8
10	5	Human	Usefulness	8
10	5	Human	Alignment	8
11	5	Large LLM	Comprehensiveness	10
11	5	Large LLM	Relevance	8
11	5	Large LLM	Specificity	9
11	5	Large LLM	Usefulness	10
11	5	Large LLM	Alignment	10
11	5	Small LLM	Comprehensiveness	8
11	5	Small LLM	Relevance	7
11	5	Small LLM	Specificity	8
11	5	Small LLM	Usefulness	7
11	5	Small LLM	Alignment	8
11	5	Human	Comprehensiveness	8
11	5	Human	Relevance	7
11	5	Human	Specificity	6
11	5	Human	Usefulness	7
11	5	Human	Alignment	7
12	5	Large LLM	Comprehensiveness	9
12	5	Large LLM	Relevance	10
12	5	Large LLM	Specificity	9
12	5	Large LLM	Usefulness	9
12	5	Large LLM	Alignment	9
12	5	Small LLM	Comprehensiveness	9
12	5	Small LLM	Relevance	9
12	5	Small LLM	Specificity	8
12	5	Small LLM	Usefulness	8
12	5	Small LLM	Alignment	8
12	5	Human	Comprehensiveness	7
12	5	Human	Relevance	8
12	5	Human	Specificity	8
12	5	Human	Usefulness	7
12	5	Human	Alignment	6
13	5	Large LLM	Comprehensiveness	7
13	5	Large LLM	Relevance	9
13	5	Large LLM	Specificity	8
13	5	Large LLM	Usefulness	8
13	5	Large LLM	Alignment	9
13	5	Small LLM	Comprehensiveness	8
13	5	Small LLM	Relevance	8
13	5	Small LLM	Specificity	7
13	5	Small LLM	Usefulness	8
13	5	Small LLM	Alignment	7
13	5	Human	Comprehensiveness	7
13	5	Human	Relevance	7
13	5	Human	Specificity	7
13	5	Human	Usefulness	8
13	5	Human	Alignment	7
14	5	Large LLM	Comprehensiveness	9
14	5	Large LLM	Relevance	8
14	5	Large LLM	Specificity	9
14	5	Large LLM	Usefulness	9
14	5	Large LLM	Alignment	9
14	5	Small LLM	Comprehensiveness	8
14	5	Small LLM	Relevance	8
14	5	Small LLM	Specificity	7
14	5	Small LLM	Usefulness	8
14	5	Small LLM	Alignment	8
14	5	Human	Comprehensiveness	7
14	5	Human	Relevance	8
14	5	Human	Specificity	7
14	5	Human	Usefulness	7
14	5	Human	Alignment	7
1	6	Large LLM	Comprehensiveness	8
1	6	Large LLM	Relevance	8
1	6	Large LLM	Specificity	8
1	6	Large LLM	Usefulness	8
1	6	Large LLM	Alignment	8
1	6	Small LLM	Comprehensiveness	7
1	6	Small LLM	Relevance	7
1	6	Small LLM	Specificity	8
1	6	Small LLM	Usefulness	7
1	6	Small LLM	Alignment	8
1	6	Human	Comprehensiveness	9
1	6	Human	Relevance	8
1	6	Human	Specificity	6
1	6	Human	Usefulness	7
1	6	Human	Alignment	8
2	6	Large LLM	Comprehensiveness	7
2	6	Large LLM	Relevance	8
2	6	Large LLM	Specificity	9
2	6	Large LLM	Usefulness	9
2	6	Large LLM	Alignment	9
2	6	Small LLM	Comprehensiveness	7
2	6	Small LLM	Relevance	8
2	6	Small LLM	Specificity	7
2	6	Small LLM	Usefulness	6
2	6	Small LLM	Alignment	7
2	6	Human	Comprehensiveness	7
2	6	Human	Relevance	7
2	6	Human	Specificity	7
2	6	Human	Usefulness	7
2	6	Human	Alignment	7
3	6	Large LLM	Comprehensiveness	8
3	6	Large LLM	Relevance	7
3	6	Large LLM	Specificity	10
3	6	Large LLM	Usefulness	9
3	6	Large LLM	Alignment	9
3	6	Small LLM	Comprehensiveness	8
3	6	Small LLM	Relevance	7
3	6	Small LLM	Specificity	7
3	6	Small LLM	Usefulness	7
3	6	Small LLM	Alignment	8
3	6	Human	Comprehensiveness	7
3	6	Human	Relevance	7
3	6	Human	Specificity	7
3	6	Human	Usefulness	7
3	6	Human	Alignment	9
4	6	Large LLM	Comprehensiveness	8
4	6	Large LLM	Relevance	9
4	6	Large LLM	Specificity	8
4	6	Large LLM	Usefulness	9
4	6	Large LLM	Alignment	9
4	6	Small LLM	Comprehensiveness	7
4	6	Small LLM	Relevance	7
4	6	Small LLM	Specificity	7
4	6	Small LLM	Usefulness	8
4	6	Small LLM	Alignment	7
4	6	Human	Comprehensiveness	6
4	6	Human	Relevance	7
4	6	Human	Specificity	7
4	6	Human	Usefulness	8
4	6	Human	Alignment	7
5	6	Large LLM	Comprehensiveness	10
5	6	Large LLM	Relevance	10
5	6	Large LLM	Specificity	9
5	6	Large LLM	Usefulness	9
5	6	Large LLM	Alignment	10
5	6	Small LLM	Comprehensiveness	9
5	6	Small LLM	Relevance	8
5	6	Small LLM	Specificity	8
5	6	Small LLM	Usefulness	7
5	6	Small LLM	Alignment	7
5	6	Human	Comprehensiveness	6
5	6	Human	Relevance	7
5	6	Human	Specificity	7
5	6	Human	Usefulness	7
5	6	Human	Alignment	8
6	6	Large LLM	Comprehensiveness	8
6	6	Large LLM	Relevance	9
6	6	Large LLM	Specificity	10
6	6	Large LLM	Usefulness	9
6	6	Large LLM	Alignment	9
6	6	Small LLM	Comprehensiveness	8
6	6	Small LLM	Relevance	7
6	6	Small LLM	Specificity	8
6	6	Small LLM	Usefulness	7
6	6	Small LLM	Alignment	7
6	6	Human	Comprehensiveness	7
6	6	Human	Relevance	8
6	6	Human	Specificity	6
6	6	Human	Usefulness	7
6	6	Human	Alignment	7
7	6	Large LLM	Comprehensiveness	10
7	6	Large LLM	Relevance	7
7	6	Large LLM	Specificity	9
7	6	Large LLM	Usefulness	9
7	6	Large LLM	Alignment	9
7	6	Small LLM	Comprehensiveness	8
7	6	Small LLM	Relevance	8
7	6	Small LLM	Specificity	6
7	6	Small LLM	Usefulness	9
7	6	Small LLM	Alignment	8
7	6	Human	Comprehensiveness	7
7	6	Human	Relevance	6
7	6	Human	Specificity	7
7	6	Human	Usefulness	8
7	6	Human	Alignment	9
8	6	Large LLM	Comprehensiveness	9
8	6	Large LLM	Relevance	9
8	6	Large LLM	Specificity	8
8	6	Large LLM	Usefulness	8
8	6	Large LLM	Alignment	10
8	6	Small LLM	Comprehensiveness	8
8	6	Small LLM	Relevance	9
8	6	Small LLM	Specificity	6
8	6	Small LLM	Usefulness	7
8	6	Small LLM	Alignment	7
8	6	Human	Comprehensiveness	7
8	6	Human	Relevance	7
8	6	Human	Specificity	7
8	6	Human	Usefulness	7
8	6	Human	Alignment	7
9	6	Large LLM	Comprehensiveness	10
9	6	Large LLM	Relevance	10
9	6	Large LLM	Specificity	10
9	6	Large LLM	Usefulness	10
9	6	Large LLM	Alignment	10
9	6	Small LLM	Comprehensiveness	7
9	6	Small LLM	Relevance	8
9	6	Small LLM	Specificity	6
9	6	Small LLM	Usefulness	8
9	6	Small LLM	Alignment	9
9	6	Human	Comprehensiveness	7
9	6	Human	Relevance	8
9	6	Human	Specificity	8
9	6	Human	Usefulness	8
9	6	Human	Alignment	8
10	6	Large LLM	Comprehensiveness	8
10	6	Large LLM	Relevance	9
10	6	Large LLM	Specificity	10
10	6	Large LLM	Usefulness	9
10	6	Large LLM	Alignment	10
10	6	Small LLM	Comprehensiveness	8
10	6	Small LLM	Relevance	8
10	6	Small LLM	Specificity	9
10	6	Small LLM	Usefulness	7
10	6	Small LLM	Alignment	8
10	6	Human	Comprehensiveness	8
10	6	Human	Relevance	7
10	6	Human	Specificity	8
10	6	Human	Usefulness	8
10	6	Human	Alignment	8
11	6	Large LLM	Comprehensiveness	10
11	6	Large LLM	Relevance	8
11	6	Large LLM	Specificity	8
11	6	Large LLM	Usefulness	10
11	6	Large LLM	Alignment	10
11	6	Small LLM	Comprehensiveness	8
11	6	Small LLM	Relevance	8
11	6	Small LLM	Specificity	7
11	6	Small LLM	Usefulness	8
11	6	Small LLM	Alignment	8
11	6	Human	Comprehensiveness	7
11	6	Human	Relevance	7
11	6	Human	Specificity	7
11	6	Human	Usefulness	6
11	6	Human	Alignment	6
12	6	Large LLM	Comprehensiveness	8
12	6	Large LLM	Relevance	8
12	6	Large LLM	Specificity	8
12	6	Large LLM	Usefulness	9
12	6	Large LLM	Alignment	9
12	6	Small LLM	Comprehensiveness	8
12	6	Small LLM	Relevance	8
12	6	Small LLM	Specificity	8
12	6	Small LLM	Usefulness	9
12	6	Small LLM	Alignment	7
12	6	Human	Comprehensiveness	6
12	6	Human	Relevance	7
12	6	Human	Specificity	6
12	6	Human	Usefulness	9
12	6	Human	Alignment	8
13	6	Large LLM	Comprehensiveness	10
13	6	Large LLM	Relevance	7
13	6	Large LLM	Specificity	8
13	6	Large LLM	Usefulness	8
13	6	Large LLM	Alignment	9
13	6	Small LLM	Comprehensiveness	6
13	6	Small LLM	Relevance	7
13	6	Small LLM	Specificity	7
13	6	Small LLM	Usefulness	6
13	6	Small LLM	Alignment	8
13	6	Human	Comprehensiveness	6
13	6	Human	Relevance	7
13	6	Human	Specificity	6
13	6	Human	Usefulness	6
13	6	Human	Alignment	7
14	6	Large LLM	Comprehensiveness	9
14	6	Large LLM	Relevance	8
14	6	Large LLM	Specificity	9
14	6	Large LLM	Usefulness	9
14	6	Large LLM	Alignment	10
14	6	Small LLM	Comprehensiveness	8
14	6	Small LLM	Relevance	6
14	6	Small LLM	Specificity	6
14	6	Small LLM	Usefulness	8
14	6	Small LLM	Alignment	8
14	6	Human	Comprehensiveness	8
14	6	Human	Relevance	7
14	6	Human	Specificity	6
14	6	Human	Usefulness	8
14	6	Human	Alignment	7
"""

# --- Parsing & cleaning ---
df = pd.read_csv(io.StringIO(raw), sep="\t")
df.columns = [c.strip().replace(" ", "_") for c in df.columns]
df["Score"] = pd.to_numeric(df["Score"], errors="coerce")
df = df.dropna(subset=["Score"])

# Composite score per Prompt_ID x Evaluator x Response_Type
group_cols = ["Prompt_ID", "Evaluator", "Response_Type"]
composite = (
    df.groupby(group_cols, as_index=False)["Score"]
      .mean()
      .rename(columns={"Score": "MeanScore"})
)

# Pivot to subjects x conditions (paired design)
subj = composite.copy()
subj["Subject"] = subj["Prompt_ID"].astype(str) + "_" + subj["Evaluator"].astype(str)
pivot = subj.pivot_table(index="Subject", columns="Response_Type", values="MeanScore")

# Keep only complete cases (all three response types present)
needed = ["Human", "Small LLM", "Large LLM"]
pivot = pivot.dropna(subset=needed)
pivot = pivot[needed]

n_subjects = pivot.shape[0]
k = len(needed)

# Friedman test (repeated-measures, nonparametric)
friedman_stat, friedman_p = stats.friedmanchisquare(
    pivot["Human"], pivot["Small LLM"], pivot["Large LLM"]
)
kendalls_W = friedman_stat / (n_subjects * (k - 1)) if n_subjects > 0 else np.nan

# Pairwise Wilcoxon (Holm)
pairs = [("Large LLM", "Small LLM"), ("Large LLM", "Human"), ("Small LLM", "Human")]
wilcoxon_results, p_vals = [], []
for a, b in pairs:
    # Wilcoxon requires paired arrays; drop any rows with NaN in either column (already filtered, but safe)
    paired = pivot[[a, b]].dropna()
    if len(paired) == 0:
        stat, p = np.nan, np.nan
    else:
        # zero_method='wilcox' avoids removing zeros incorrectly; if all diffs=0, scipy may raise a warning—catch gracefully
        try:
            stat, p = stats.wilcoxon(paired[a], paired[b], zero_method="wilcox",
                                     correction=False, alternative="two-sided", mode="auto")
        except ValueError:
            stat, p = np.nan, np.nan
    p_vals.append(p)
    wilcoxon_results.append({"Pair": f"{a} vs {b}", "W": stat, "p_raw": p})

# Holm-Bonferroni
def holm_correction(p_list):
    idx = np.argsort([1 if (np.isnan(p)) else p for p in p_list])  # NaNs go last
    m = len(p_list)
    adj = [np.nan] * m
    for rank, i in enumerate(idx, start=1):
        p = p_list[i]
        if np.isnan(p):
            adj[i] = np.nan
        else:
            adj[i] = min(p * (m - rank + 1), 1.0)
    return adj

adj = holm_correction(p_vals)
for i, res in enumerate(wilcoxon_results):
    res["p_holm"] = adj[i]
wilcoxon_df = pd.DataFrame(wilcoxon_results)

# Kruskal–Wallis on composite scores (ignoring pairing)
kw_groups = [composite.loc[composite["Response_Type"] == rt, "MeanScore"] for rt in needed]
kw_stat, kw_p = stats.kruskal(*kw_groups)
N_kw = sum(len(g) for g in kw_groups)

# Epsilon-squared (η²_H) approximation for Kruskal–Wallis
eta_sq_H = (kw_stat - (k - 1)) / (N_kw - k) if (N_kw - k) > 0 else np.nan

# Summaries
summary = pd.DataFrame({
    "Test": ["Friedman (RM nonparametric)", "Kendall's W (effect size)",
             "Kruskal–Wallis (between-groups)", "Eta^2_H (effect size)"],
    "Statistic": [friedman_stat, kendalls_W, kw_stat, eta_sq_H],
    "p_value": [friedman_p, np.nan, kw_p, np.nan],
    "n": [n_subjects, n_subjects, N_kw, N_kw]
})

# Group means (composite)
means = (composite.groupby("Response_Type")["MeanScore"]
         .agg(mean="mean", std="std", count="count")
         .reset_index())

# --- Save outputs ---
summary.round(4).to_csv("nonparametric_summary.csv", index=False)
wilcoxon_df.round(4).to_csv("wilcoxon_signedrank_holm.csv", index=False)
pd.DataFrame({
    "Pair": [r["Pair"] for r in wilcoxon_results],
    "W": [r["W"] for r in wilcoxon_results],
    "p_raw": [r["p_raw"] for r in wilcoxon_results],
    "p_holm": [r["p_holm"] for r in wilcoxon_results],
}).round(4).to_csv("wilcoxon_signedrank_holm.csv", index=False)
pd.DataFrame({
    "Response_Type": ["Human", "Small LLM", "Large LLM"],
    "MeanScore_mean": [means.loc[means["Response_Type"] == rt, "mean"].values[0] for rt in needed],
    "MeanScore_std": [means.loc[means["Response_Type"] == rt, "std"].values[0] for rt in needed],
    "n": [means.loc[means["Response_Type"] == rt, "count"].values[0] for rt in needed],
}).round(3).to_csv("composite_means_by_response_type.csv", index=False)

print("DONE")
print(summary.round(4))
print("\nWilcoxon (paired, Holm-adjusted):")
print(wilcoxon_df.round(4))
print("\nComposite means by response type:")
print(means.round(3))

DONE
                              Test  Statistic  p_value    n
0      Friedman (RM nonparametric)    27.7855      0.0   84
1        Kendall's W (effect size)     0.1654      NaN   84
2  Kruskal–Wallis (between-groups)    52.8713      0.0  252
3            Eta^2_H (effect size)     0.2043      NaN  252

Wilcoxon (paired, Holm-adjusted):
                     Pair      W   p_raw  p_holm
0  Large LLM vs Small LLM  515.5  0.0000  0.0000
1      Large LLM vs Human  557.5  0.0000  0.0000
2      Small LLM vs Human  724.5  0.0002  0.0002

Composite means by response type:
  Response_Type   mean    std  count
0         Human  7.383  1.091     84
1     Large LLM  8.417  1.250     84
2     Small LLM  7.807  0.776     84
